In [1]:
from networkx.algorithms import bipartite
import networkx as nx
from networkx import *
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [2]:
# 80000row
user_train = pd.read_csv('ml-100k/u5.base', sep='\t',names=["userID","itemID","rating","timestamp"],header=None, na_filter=False)
user_train = user_train[['userID','itemID','rating']]

# 100000row
user_total = pd.read_csv('ml-100k/u.data', sep='\t',names=["userID","itemID","rating","timestamp"],header=None, na_filter=False)
user_total = user_total[['userID','itemID','rating']]

# 20000row
user_test = pd.read_csv('ml-100k/u5.test', sep='\t',names=["userID","itemID","rating","timestamp"],header=None, na_filter=False)
user_test = user_test[['userID','itemID','rating']]


In [3]:
'''
min_max_scaler = MinMaxScaler()

x_scaled = min_max_scaler.fit_transform(user_train[['rating']])
user_train[['rating']] = x_scaled

x_scaled = min_max_scaler.fit_transform(user_total[['rating']])
user_total[['rating']] = x_scaled

x_scaled = min_max_scaler.fit_transform(user_test[['rating']])
user_test [['rating']] = x_scaled
'''

"\nmin_max_scaler = MinMaxScaler()\n\nx_scaled = min_max_scaler.fit_transform(user_train[['rating']])\nuser_train[['rating']] = x_scaled\n\nx_scaled = min_max_scaler.fit_transform(user_total[['rating']])\nuser_total[['rating']] = x_scaled\n\nx_scaled = min_max_scaler.fit_transform(user_test[['rating']])\nuser_test [['rating']] = x_scaled\n"

In [4]:
user_item_total = user_total.pivot_table('rating', index = 'userID',columns = 'itemID').fillna(0)
user_item_train = user_train.pivot_table('rating', index = 'userID',columns = 'itemID').fillna(0) # train 용
user_item_test = user_test.pivot_table('rating', index = 'userID',columns = 'itemID').fillna(0)

total_user_idx = user_item_total.index

In [5]:
## user-item node 이름 설정

matrix = pd.read_csv('ml-100k/u.data', sep='\t',names=["userID","itemID","rating","timestamp"],header=None, na_filter=False)
matrix[['userID']] = 'u' + matrix[['userID']].astype(str)
matrix[['itemID']] = 'i' + matrix[['itemID']].astype(str)
matrix

,userID,itemID,rating,timestamp
0,u196,i242,3,881250949
1,u186,i302,3,891717742
2,u22,i377,1,878887116
3,u244,i51,2,880606923
4,u166,i346,1,886397596
...,...,...,...,...
99995,u880,i476,3,880175444
99996,u716,i204,5,879795543
99997,u276,i1090,1,874795795
99998,u13,i225,2,882399156


In [6]:
## user-item node 이름 설정

matrix = pd.read_csv('ml-100k/u.data', sep='\t',names=["userID","itemID","rating","timestamp"],header=None, na_filter=False)
matrix[['userID']] = 'u' + matrix[['userID']].astype(str)
matrix[['itemID']] = 'i' + matrix[['itemID']].astype(str)
matrix

,userID,itemID,rating,timestamp
0,u196,i242,3,881250949
1,u186,i302,3,891717742
2,u22,i377,1,878887116
3,u244,i51,2,880606923
4,u166,i346,1,886397596
...,...,...,...,...
99995,u880,i476,3,880175444
99996,u716,i204,5,879795543
99997,u276,i1090,1,874795795
99998,u13,i225,2,882399156


In [7]:
import numpy as np
user_node = matrix[['userID']].values
user_node = np.array(user_node).flatten().tolist()
#user_node

In [8]:
G = nx.Graph()
G.add_nodes_from(user_node)
info(G)

C:\Users\kddin\AppData\Local\Temp\ipykernel_11704\1259828340.py:3: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  info(G)


'Graph with 943 nodes and 0 edges'

In [9]:
edgelist = []

for i in matrix.values:
    edgelist.append((i[0],i[1]))
    
B = nx.Graph() # >300
B.add_nodes_from(matrix.userID, bipartite=0) # user
B.add_nodes_from(matrix.itemID, bipartite=1) # item(movie)
B.add_edges_from(edgelist)

info(B)

C:\Users\kddin\AppData\Local\Temp\ipykernel_11704\700776426.py:11: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  info(B)


'Graph with 2625 nodes and 100000 edges'

## Link Prediction

 전체 그래프에서 상위 1%의 연결 가능성을 해당하는 link connection

In [10]:
for i in range(10):
    
    user_edgelist = []
    total_edgelist = []
    
    pred_link = list(nx.common_neighbor_centrality(B, alpha=0.8))
    
    link_likelihood = pd.DataFrame(pred_link).iloc[:,2].values
    link_99 = np.percentile(link_likelihood,  99.5, interpolation='linear')
    
    for p in pred_link:
        edge = list(p)
        linklihood = int(edge[2])
        
        if linklihood > link_99:
            if edge[0][:1] ==  edge[1][:1] == 'u': # user R
                user_edgelist.append((edge[0], edge[1]))
                total_edgelist.append((edge[0], edge[1]))
            else :
                total_edgelist.append((edge[0], edge[1]))
                
    B.add_edges_from(total_edgelist)
    G.add_edges_from(user_edgelist)
    
    print('total_graph connected : ', is_connected(B),' user graph info : ', info(B))
    print('user_graph connected : ', is_connected(G),' user graph info : ', info(G))
    
    if is_connected(B):
        break
    
#print('total graph: ', info(B))
#print('user_graph connected : ', is_connected(G),' user graph info : ', info(G))

C:\Users\kddin\AppData\Local\Temp\ipykernel_11704\1209186748.py:9: DeprecationWarning: the `interpolation=` argument to percentile was renamed to `method=`, which has additional options.
Users of the modes 'nearest', 'lower', 'higher', or 'midpoint' are encouraged to review the method they. (Deprecated NumPy 1.22)
  link_99 = np.percentile(link_likelihood,  99.5, interpolation='linear')


total_graph connected :  True  user graph info :  Graph with 2625 nodes and 116425 edges
user_graph connected :  False  user graph info :  Graph with 943 nodes and 8702 edges


C:\Users\kddin\AppData\Local\Temp\ipykernel_11704\1209186748.py:25: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print('total_graph connected : ', is_connected(B),' user graph info : ', info(B))
C:\Users\kddin\AppData\Local\Temp\ipykernel_11704\1209186748.py:26: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print('user_graph connected : ', is_connected(G),' user graph info : ', info(G))


## Graph Clustering

In [11]:
from networkx.algorithms.community import k_clique_communities
from networkx.algorithms.community import coverage, performance

cluster = []
total_graph = B

In [12]:
communities = k_clique_communities(total_graph, k=8)

In [13]:
node_groups = []

In [ ]:
for com in next(communities):
    node_groups.append(list(com))
    
print(node_groups)

In [ ]:
comp = k_clique_communities(total_graph,k=3)

In [ ]:
result = {frozenset(c) for c in comp}

In [ ]:
reulst

In [ ]:
len(result)